<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EC%9D%91%EC%9A%A9(%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5)_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [ ]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
import torch
dataset = load_dataset('klue','mrc')
model_name = "monologg/kobert"
model_QA_FE = AutoModel.from_pretrained(model_name)
tokenizer_QA_FE = AutoTokenizer.from_pretrained(model_name)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_QA_FE.to(device)
model_QA_FE.eval()

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
# 임베딩 추출함수
import torch
import numpy as np
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def extract_cls_embedding(dataset, tokenizer, model, max_samplings=1000):
  embeddings = []
  labels = []  # 1 답변가능, 0 답변 불가능
  for i ,example in tqdm(enumerate(dataset)):
    if i>= max_samplings:
      break
    question = example['question']
    context = example['context']
    answer = example['answers']['text'][0]
    label = 1 if answer else 0
    # 토큰화
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True,
                       padding='max_length', max_length=384).to(device)
    # cls 임베딩 추출
    with torch.no_grad():
      outputs = model(**inputs)
      cls_embedding = outputs.last_hidden_state[:,0,:].cpu().numpy()
      embeddings.append(cls_embedding[0])  # cls 임베딩
      labels.append(label)
  return np.array(embeddings), np.array(labels)

In [ ]:
# 임베딩 추출
train_embeddings, train_labels = extract_cls_embedding(dataset['train'],tokenizer_QA_FE,model_QA_FE)
test_embeddings, test_labels = extract_cls_embedding(dataset['validation'],tokenizer_QA_FE,model_QA_FE
                                                     ,max_samplings=200)
# 머신러닝 학습
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(max_iter=10)
model.fit(train_embeddings, train_labels)
model.score(train_embeddings, train_labels)  # accuracy

0it [00:00, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
1it [00:02,  2.16s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
2it [00:04,  2.13s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
3it [00:06,  2.12s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
4it [00:08, 

KeyboardInterrupt: 

Fine tuning

In [ ]:
!pip install datasets -q
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
devcie = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = load_dataset("klue", "mrc")
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(devcie)
def preprocess_function(examples):
    questions =  examples['question']
    contexts =  examples['context']
    answers =  examples['answers']
    tokenized = tokenizer(
        questions,
        contexts,
        max_length = 384,
        truncation = 'only_second',
        padding = 'max_length',
        return_offsets_mapping = True,
    )
    # 시작과 끝레이블 생성
    start_positions = []
    end_positions     = []
    for i in range(len(answers)):
      answer = answers[i]
      start_char = answer['answer_start'][0]
      end_char = start_char + len(answer['text'][0])
      offset_mapping = tokenized['offset_mapping'][i]
      token_start = None
      token_end = None
      for j, (start,end) in enumerate(offset_mapping):
        if start == start_char:
          token_start = j
        if end == end_char:
          token_end = j
          break
      start_positions.append(token_start if token_start is not None else 0)
      end_positions.append(token_end if token_end is not None else 0)
    tokenized['start_positions'] = start_positions
    tokenized['end_positions'] = end_positions
    return tokenized

# 데이터 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [ ]:
# train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(3000))
train_dataset = tokenized_datasets['train'].shuffle(seed=42)
eval_dataset = tokenized_datasets['validation'].shuffle(seed=42).select(range(200))

In [ ]:
# 평가 매트릭스
def compute_metrics(eval_pred):
  predictions, labels =  eval_pred
  start_logits, end_logits = predictions
  start_positions, end_positions = labels

  start_pred = np.argmax(start_logits, axis=1)
  end_pred = np.argmax(end_logits, axis=1)

  exact_match = np.mean( (start_pred == start_positions) & (end_pred == end_positions))
  return {'exact_match' : exact_match}

In [ ]:
# 학습설정
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir='./result', report_to="none",num_train_epochs=3)
trainer = Trainer(model=model, args=training_args
                  ,train_dataset=train_dataset
                  ,eval_dataset=eval_dataset
                  ,compute_metrics=compute_metrics
                  )
trainer.train()
model.save_pretrained('./result')
tokenizer.save_pretrained('./result')

Step,Training Loss
500,1.752000
1000,1.633700
1500,1.562900
2000,1.487500
2500,1.041100
3000,0.823000
3500,0.806600
4000,0.830600
4500,0.680300
5000,0.378100


('./result/tokenizer_config.json',
 './result/special_tokens_map.json',
 './result/vocab.txt',
 './result/added_tokens.json',
 './result/tokenizer.json')

In [ ]:
# 추론 함수 생성
print(dataset['validation']['context'][0])
print(dataset['validation']['question'][0])
print(dataset['validation']['answers'][0]['text'])

BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
말라카이트에서 나온 색깔을 사용한 에디션은?
['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']


In [ ]:
# 추론 함수
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def predict_asnwer(context, question, model, tokenizer):
  model.eval()
  inputs = tokenizer(question, context, max_length=384
                     , truncation='only_second'
                     , padding='max_length'
                     , return_tensors='pt').to(device)
  with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)
    # 답변 디코딩
    answer_tokens = inputs['input_ids'][0][start_idx:end_idx+1]
    answer = tokenizer.decode(answer_tokens,skip_special_tokens=True)
  return answer

In [ ]:
context = "현대자동차는 다양한 자동차를 생산하고 있으며, 대표 차량으로는 쏘나타가 있다"
question = [
    '현대자동차는 어떤 차를 생산하나요?',
    '현대자동차의 대표 차량은?'
]
for q in question:
  answer = predict_asnwer(context, q, model, tokenizer)
  print(answer)


다양한 자동차를 생산하고 있으며, 대표 차량으로는 쏘나타
쏘나타


In [ ]:
context = dataset['validation']['context'][0]
question = dataset['validation']['question'][0]
predict_asnwer(context, question, model, tokenizer)

'뉴 740Li 25주년 에디션'